In [ ]:
nearby_hotels_location = nearby_hotels[["hotel", "latitude", "longitude"]] # 최소 소요 시간을 위한 dataframe
nearby_hotels_location

,hotel,latitude,longitude
91,대전 유성 뉴큐피트,36.356338,127.337533
238,유성 뉴큐피트,36.356338,127.337533
258,유성 스테이호텔,36.356027,127.337611
104,대전 유성 스테이,36.356027,127.337611
98,대전 유성 발렌타인,36.357106,127.337561
250,유성 발렌타인,36.357106,127.337561
271,유성 피카소,36.356348,127.337704
247,유성 만월 호텔-한옥,36.357142,127.337842
242,유성 드라마,36.354536,127.339684
263,유성 원,36.356086,127.340177


In [ ]:
import requests

# 네이버클라우드 API 정보
#Client ID 입력
#(X-NCP-APIGW-API-KEY-ID)
dir_client_id = "5umre9k7pr"
#Client Secret 입력
#(X-NCP-APIGW-API-KEY)
dir_client_secret = "tLNrO5glKtPLHnJC8tE59TR2KIRP33Vgz6TX8yiA"

url = "https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving"
headers = {
    "X-NCP-APIGW-API-KEY-ID": dir_client_id,
    "X-NCP-APIGW-API-KEY": dir_client_secret,
}

hotel_times = []  # 호텔 이름과 예상 시간을 저장할 리스트

for index, row in nearby_hotels_location.iterrows():
    start = f"{user_location[1]},{user_location[0]}"  # 출발지 (사용자의 경도, 위도)
    goal = f"{row['longitude']},{row['latitude']}"  # 목표지 (도착지의 경도, 위도)

    # 요청 파라미터, 언어 기본값 한국어
    params = {
        "start": start,
        "goal": goal,
        "option": "trafast",  # 실시간 빠른 길
    }

    # API 요청
    response = requests.get(url, headers=headers, params=params)

    # 결과
    if response.status_code == 200: # 코드가 정상 동작했다면 200이라는 값을 줌
        result = response.json() # json 형태로 저장
        if "route" in result: # route는 경로 조회 결과. route가 없으면 세부 정보 확인 불가능
            route_data = result["route"].get("trafast", []) # 최대한 빠른 경로에 대한 시간 확인
            if route_data:
                # 첫 번째 경로에 대한 정보 확인
                duration = route_data[0].get("summary", {}).get("duration", None) #  route.{summary.duration}이 소요 시간에 대한 정보(단위 : 밀리초)
                if duration:
                    # 예상 시간을 밀리초에서 분으로 변환 후 반올림하여 정수로 출력
                    duration_minutes_rounded = round(duration / 60000)
                    # 호텔 이름과 예상 시간을 저장
                    hotel_times.append({"hotel": row['hotel'], "estimated_time": f"{duration_minutes_rounded} minutes"}) # hotel_times에 호텔 이름, 걸린 시간 append
                else:
                    hotel_times.append({"hotel": row['hotel'], "estimated_time": "Duration data not available"})    # route 내 시간 X
            else:
                hotel_times.append({"hotel": row['hotel'], "estimated_time": "No route information available"}) # 실시간 빠른 길 X
        else:
            hotel_times.append({"hotel": row['hotel'], "estimated_time": "Route information not found in the response"}) # route X
    else:
        hotel_times.append({"hotel": row['hotel'], "estimated_time": f"Error with status {response.status_code}"}) # API 요청이 정상적으로 이루어지지 않음

# 결과를 데이터프레임으로 변환
hotel_times_df = pd.DataFrame(hotel_times)

# 결과 확인
print(hotel_times_df)


         hotel estimated_time
0   대전 유성 뉴큐피트      8 minutes
1      유성 뉴큐피트      8 minutes
2     유성 스테이호텔      8 minutes
3    대전 유성 스테이      8 minutes
4   대전 유성 발렌타인      8 minutes
5      유성 발렌타인      8 minutes
6       유성 피카소      8 minutes
7  유성 만월 호텔-한옥      8 minutes
8       유성 드라마      9 minutes
9         유성 원      9 minutes
